In [ ]:
!pip install TTS


In [ ]:
!pip install numpy==1.26.4 --force-reinstall
# Dopo aver eseguito questa cella riavviare il runtime !!!!

In [ ]:

import os
import json
from TTS.api import TTS
from tqdm import tqdm
from google.colab import files
import random

uploaded = files.upload()

In [ ]:
sentences_filename = "sentences.json"

if sentences_filename not in uploaded:
    print(f"Errore: Il file '{sentences_filename}' non è stato caricato.")
else:
    print(f"Caricamento di '{sentences_filename}' completato.")
    with open(sentences_filename, "r", encoding="utf-8") as f:
        # Ora all_sentences è una LISTA DI STRINGHE
        all_sentences = json.load(f)
        print(f"Caricate {len(all_sentences)} frasi.")

    half = len(all_sentences) // 2
    sentences_split = {
        "tts_models/it/mai_female/glow-tts": {
            "sentences": all_sentences[:half], # Prima metà delle frasi (lista di stringhe)
            "gender": "female"
        },
        "tts_models/it/mai_male/glow-tts": {
            "sentences": all_sentences[half:], # Seconda metà delle frasi (lista di stringhe)
            "gender": "male"
        }
    }

    base_output_dir = "coqui_output"
    os.makedirs(base_output_dir, exist_ok=True)

    all_final_samples_metadata = []

    for model_name, data in sentences_split.items():
        sentences_list = data["sentences"]
        gender = data["gender"]
        subfolder = os.path.join(base_output_dir, gender)
        os.makedirs(subfolder, exist_ok=True)

        print(f"\nInizializzazione modello Coqui TTS: {model_name}...")
        try:
             tts = TTS(model_name=model_name, progress_bar=True)
             print(f"Modello {model_name} caricato.")
        except Exception as e:
             print(f"Errore durante il caricamento del modello {model_name}: {e}")
             continue

        current_split_metadata = []

        print(f"Generazione audio per {len(sentences_list)} frasi ({gender})...")
        for i, sentence_text in tqdm(enumerate(sentences_list), total=len(sentences_list), desc=f"Processing {gender}"):
            text = sentence_text
            global_index = i if gender == "female" else i + half

            output_filename = f"sample_{global_index}_{gender}.wav"
            output_path = os.path.join(subfolder, output_filename)

            try:
                tts.tts_to_file(text=text, file_path=output_path)

                sample_metadata = {
                    "id": global_index,
                    "text": text,
                    "coqui_audio_path": os.path.join(gender, output_filename),
                    "coqui_model": model_name,
                    "speaker_gender": gender
                }
                current_split_metadata.append(sample_metadata)

            except Exception as e_tts:
                print(f"\nErrore durante la sintesi per la frase {global_index} ({gender}): {e_tts}")

        metadata_split_path = os.path.join(subfolder, "samples_coqui.json")
        try:
            with open(metadata_split_path, "w", encoding="utf-8") as f:
                json.dump(current_split_metadata, f, indent=2, ensure_ascii=False)
            print(f"Metadati per '{gender}' salvati in: {metadata_split_path}")
            all_final_samples_metadata.extend(current_split_metadata)
        except Exception as e_save:
            print(f"Errore durante il salvataggio dei metadati per '{gender}': {e_save}")


    global_metadata_path = os.path.join(base_output_dir, "all_samples_coqui.json")
    try:
        all_final_samples_metadata.sort(key=lambda x: x['id'])
        with open(global_metadata_path, "w", encoding="utf-8") as f:
            json.dump(all_final_samples_metadata, f, indent=2, ensure_ascii=False)
        print(f"\nMetadati globali aggregati salvati in: {global_metadata_path}")
    except Exception as e_save_global:
         print(f"Errore durante il salvataggio dei metadati globali: {e_save_global}")


    print("\nCreazione file zip...")
    zip_filename = "coqui_output.zip"
    !zip -r {zip_filename} {base_output_dir}
    print(f"File '{zip_filename}' creato. Avvio download...")
    files.download(zip_filename)
    print("Download completato.")

if sentences_filename not in uploaded:
     print("\nOperazione interrotta perché il file sentences.json non è stato caricato.")